In [120]:
import requests
import pandas as pd

import datetime

### Useful LINK
#### https://www.kaggle.com/code/svendaj/extracting-data-from-sec-edgar-restful-apis

## 1. Get Comnpany related report list

In [77]:
cik = '0000100493'
url = f"https://data.sec.gov/submissions/CIK{cik}.json"

In [78]:
header = {
  "User-Agent": "zs.nagy.1989@gmail.com"#, # remaining fields are optional
#    "Accept-Encoding": "gzip, deflate",
#    "Host": "data.sec.gov"
}

In [79]:
company_filings = requests.get(url, headers=header).json()
company_filings.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [80]:
company_filings_df = pd.DataFrame(company_filings["filings"]["recent"])
company_filings_df.head()

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000100493-23-000096,2023-09-21,2023-09-19,2023-09-21T14:19:18.000Z,,3/A,,,,4056,0,0,xslF345X02/wk-form3a_1695320349.xml,FORM 3/A
1,0000100493-23-000094,2023-09-20,2023-09-19,2023-09-20T11:13:03.000Z,,3,,,,282547,0,0,xslF345X02/wk-form3_1695222773.xml,FORM 3
2,0000100493-23-000092,2023-09-07,2023-09-05,2023-09-07T08:00:46.000Z,34,8-K,001-14704,231241001,5.02,155227,1,1,tsn-20230905.htm,8-K
3,0000100493-23-000089,2023-08-10,2023-07-01,2023-08-10T07:50:17.000Z,34,10-Q,001-14704,231157224,,12538579,1,1,tsn-20230701.htm,10-Q
4,0000100493-23-000086,2023-08-09,2023-08-07,2023-08-09T13:15:17.000Z,,4,,,,6481,0,0,xslF345X05/wk-form4_1691601307.xml,FORM 4


In [81]:
doc_type = ['10-Q', '10-K']
company_filings_df_filt = company_filings_df.loc[company_filings_df['form'].isin(doc_type)]
# reorder the documents
company_filings_df_filt = company_filings_df_filt.reindex(index=company_filings_df_filt.index[::-1])
# drop original index
company_filings_df_filt.reset_index(inplace=True, drop=True)

In [82]:
company_filings_df_filt.head()

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001193125-10-265708,2010-11-22,2010-10-02,2010-11-22T07:59:49.000Z,34,10-K,001-14704,101207228,,32392546,1,0,d10k.htm,FORM 10-K
1,0001193125-11-024082,2011-02-04,2011-01-01,2011-02-04T11:26:12.000Z,34,10-Q,001-14704,11573095,,13799667,1,0,d10q.htm,FORM 10-Q
2,0001193125-11-131527,2011-05-09,2011-04-02,2011-05-09T07:46:22.000Z,34,10-Q,001-14704,11821388,,17129889,1,0,d10q.htm,FORM 10-Q
3,0001193125-11-213296,2011-08-08,2011-07-02,2011-08-08T07:43:04.000Z,34,10-Q,001-14704,111015661,,13900916,1,0,d10q.htm,FORM 10-Q
4,0001193125-11-317791,2011-11-21,2011-10-01,2011-11-21T07:47:25.000Z,34,10-K,001-14704,111217750,,25066045,1,0,d232196d10k.htm,FORM 10-K


## 2. Accessing specific data

In [86]:
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"

In [91]:
company_facts = requests.get(url, headers=header).json()

company_facts.keys()

dict_keys(['cik', 'entityName', 'facts'])

In [92]:
company_facts['facts'].keys()

dict_keys(['dei', 'us-gaap'])

In [102]:
company_facts['facts']['us-gaap'].keys()

dict_keys(['AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccruedIncomeTaxesCurrent', 'AccruedMarketingCostsCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossDefinedBenefitPensionAndOtherPostretirementPlansNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapital', 'AdjustmentForAmortization', 'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesOther', 'AllowanceForDoubtfulAccountsReceivable', 'AmortizationOfIntangibleAssets', 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'AociLossCashFlowHedgeCumulativeGainLossAfterTax', 'AssetImpair

In [127]:
# create DataFrame & filter
curr_assets_df = pd.DataFrame(company_facts["facts"]["us-gaap"]["AssetsCurrent"]["units"]["USD"])
curr_assets_df = curr_assets_df.loc[curr_assets_df['form'] == '10-Q']
# filter duplicates (revisioned rows)
curr_assets_df.drop_duplicates(subset=['end'], keep='last', inplace=True, ignore_index=True)
curr_assets_df['end'] = pd.to_datetime(curr_assets_df['end'])
curr_assets_df.head()

,end,val,accn,fy,fp,form,filed,frame
0,2009-10-03,4375000000,0001193125-10-182490,2010,Q3,10-Q,2010-08-09,NaN
1,2010-07-03,4364000000,0001193125-10-182490,2010,Q3,10-Q,2010-08-09,CY2010Q2I
2,2010-10-02,4618000000,0001193125-11-213296,2011,Q3,10-Q,2011-08-08,NaN
3,2011-01-01,4953000000,0001193125-11-024082,2011,Q1,10-Q,2011-02-04,CY2010Q4I
4,2011-04-02,4937000000,0001193125-11-131527,2011,Q2,10-Q,2011-05-09,CY2011Q1I


CSianl egy mampper dictionaryt az adott cikhez
- Cik key
    - tablazatom eleme subkey amihez megy a us-gaap